(d) Considere o caso particular do modelo acima com n = 50, p = 1, Xi,1 ∼ N (0, 1) e ε ∼ N (0, Σ)
independente com Σ diagonal com diagonal Σii = σ
2
i = 10
i−20
5 e β0 = 1, β1 = 0.25. Vamos
gerar amostras deste modelo e comparar o desempenho de βˆΣ com o estimador de mínimos
quadrados usual βˆ
ls
.

In [9]:
import numpy as np
from scipy import stats

n = 50
Sigma = np.diag([10**((i-20)/5) for i in range(1,n+1)])
np.random.seed(0)
X = np.array([np.ones(n),np.random.normal(0,1,n)]).T
beta = np.array([1,0.25])
epsilon = np.random.multivariate_normal(np.zeros(n),Sigma)
y = X @ beta + epsilon

(d-ii) Calcule $\hat{\beta}_{ls}$ e $\hat{\beta}_{\Sigma}$ para o conjunto de dados acima e compare como ambos estimadores se saem em termos de $\|\beta - \hat{\beta}\|_2^2$.




In [11]:
import numpy as np
import statsmodels.api as sm

ols = sm.OLS(y, X).fit()
print(ols.params)
print(ols.summary())


w = 1.0 / np.diag(Sigma)
wls = sm.WLS(y, X, weights=w).fit()
print(wls.params)
print(wls.summary())


[-34.46344733   6.94756948]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.019
Method:                 Least Squares   F-statistic:                    0.1095
Date:                Mon, 19 Jan 2026   Prob (F-statistic):              0.742
Time:                        10:52:36   Log-Likelihood:                -325.86
No. Observations:                  50   AIC:                             655.7
Df Residuals:                      48   BIC:                             659.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -34.4634   

In [15]:
# beta least squares

beta_ls = np.linalg.inv(X.T @ X) @ X.T @ y

print('Beta_ls = ',beta_ls)

# beta sigma
A = X.T @ np.linalg.inv(Sigma) @ X
beta_sig = np.linalg.inv(A) @ X.T @ np.linalg.inv(Sigma) @ y
print('Beta_sigma = ',beta_sig)

Beta_ls =  [-34.46344733   6.94756948]
Beta_sigma =  [1.01885254 0.24436202]


In [4]:
beta_ls, *_ = np.linalg.lstsq(X, y, rcond=None)
beta_ls

array([-34.46344733,   6.94756948])

In [13]:
sum_ls = np.sum((beta - beta_ls)**2)
sum_sig = np.sum((beta - beta_sig)**2)

print(sum_ls, sum_sig)


1302.5135337720312 0.000387205026253018


(d-iii) Calcule o p-valor do teste t para o coeficiente (βls) 0 como se estivéssemos supondo erroneamente que os dados foram gerados com um ε vetor de resíduos com Σ = σ^2I. Com base neste p-valor, você rejeitaria a hipótese nula?

In [19]:
X.shape[1]

2

In [26]:
p = X.shape[1]
y_hat = X@beta_ls

sigma2_hat = (1/(n - p - 1))*np.sum((y - y_hat)**2)

Sigma_linha = sigma2_hat * np.identity(n = n)

# Para testar H0: bo = 0:

t0 = beta_ls[0]/np.sqrt(sigma2_hat * np.linalg.inv(X.T@X)[0,0])

df = n - p - 1

p_bilateral = 2*(1 - stats.t.cdf(abs(t0), df))

p_bilateral

np.float64(0.15876753326459347)

In [28]:
Z = beta_sig[0]/np.sqrt(np.linalg.inv(X.T @ np.linalg.inv(Sigma) @ X)[1,1])
Z

np.float64(105.75329561203317)

In [29]:
from scipy.stats import norm

p_valor = 2*(1 - norm.cdf(abs(Z)))
p_valor

np.float64(0.0)

In [30]:
norm.cdf(abs(Z))

np.float64(1.0)